In [1]:
import sqlite3

In [2]:
# Create a connection to SQLite database
conn = sqlite3.connect('nvidia_database.db')
cursor = conn.cursor()

In [3]:
# 1. Create tables
cursor.executescript('''
-- Create products table
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    category TEXT NOT NULL,
    release_date DATE,
    price REAL
);

-- Create customers table
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT NOT NULL,
    industry TEXT,
    contact_email TEXT,
    contact_phone TEXT
);

-- Create sales table
CREATE TABLE IF NOT EXISTS sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    customer_id INTEGER,
    sale_date DATE,
    region TEXT,
    quantity_sold INTEGER,
    revenue REAL,
    FOREIGN KEY (product_id) REFERENCES products (product_id),
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);

-- Create suppliers table
CREATE TABLE IF NOT EXISTS suppliers (
    supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
    supplier_name TEXT NOT NULL,
    material_supplied TEXT NOT NULL,
    contact_email TEXT
);

-- Create supply_chain table
CREATE TABLE IF NOT EXISTS supply_chain (
    supply_chain_id INTEGER PRIMARY KEY AUTOINCREMENT,
    supplier_id INTEGER,
    product_id INTEGER,
    supply_date DATE,
    quantity_supplied INTEGER,
    FOREIGN KEY (supplier_id) REFERENCES suppliers (supplier_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
);

-- Create departments table
CREATE TABLE IF NOT EXISTS departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT,
    department_name TEXT NOT NULL,
    location TEXT
);

-- Create employees table
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department_id INTEGER,
    hire_date DATE,
    salary REAL,
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
);

-- Create projects table
CREATE TABLE IF NOT EXISTS projects (
    project_id INTEGER PRIMARY KEY AUTOINCREMENT,
    project_name TEXT NOT NULL,
    department_id INTEGER,
    start_date DATE,
    end_date DATE,
    budget REAL,
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
);
''')

# Commit table creation
conn.commit()

In [4]:
# 2. Populate tables with sample data
# Define sample data for each table
products_data = [
    ('RTX 4090', 'GPU', '2023-01-15', 1599.99),
    ('RTX 4080', 'GPU', '2022-11-16', 1199.99),
    ('RTX 4070', 'GPU', '2023-03-08', 599.99),
    ('Jetson Nano', 'AI Accelerator', '2022-05-20', 99.99),
    ('Jetson AGX Orin', 'AI Accelerator', '2023-06-10', 1999.99),
    ('DGX Station', 'Server', '2023-08-01', 8999.99),
    ('A100 Tensor Core', 'AI Accelerator', '2021-07-15', 14999.99),
    ('H100 Tensor Core', 'AI Accelerator', '2022-10-10', 19999.99),
    ('GeForce GTX 1660', 'GPU', '2019-02-22', 279.99),
    ('Titan V', 'GPU', '2017-12-07', 2999.99)
]

customers_data = [
    ('Tech Corp', 'Gaming', 'contact@techcorp.com', '123-456-7890'),
    ('AI Labs', 'Research', 'info@ailabs.org', '987-654-3210'),
    ('Data Solutions', 'Data Centers', 'support@datasolutions.com', '555-678-9101'),
    ('GameZone', 'Gaming', 'support@gamezone.net', '789-456-1230'),
    ('DeepVision', 'AI Research', 'info@deepvision.ai', '123-123-1234'),
    ('ComputePro', 'Data Centers', 'contact@computepro.com', '321-321-4321'),
    ('E-Sports World', 'Gaming', 'support@esportsworld.com', '654-987-6543'),
    ('Quantum AI', 'Research', 'info@quantumai.net', '555-123-4567'),
    ('Pixel Masters', 'Gaming', 'support@pixelmasters.com', '789-789-7890'),
    ('Global Cloud', 'Data Centers', 'support@globalcloud.com', '987-987-9876')
]

sales_data = [
    (1, 1, '2024-01-10', 'North America', 100, 159999.00),
    (2, 4, '2024-01-12', 'Europe', 200, 239998.00),
    (3, 5, '2024-01-15', 'Asia', 150, 89998.50),
    (4, 2, '2024-01-20', 'North America', 300, 29997.00),
    (5, 3, '2024-01-25', 'Europe', 80, 159999.20),
    (6, 6, '2024-01-30', 'Asia', 20, 179999.80)
]

suppliers_data = [
    ('SiliconPro', 'Silicon Wafers', 'sales@siliconpro.com'),
    ('ChipCo', 'Transistors', 'info@chipco.net'),
    ('NanoMaterials', 'Copper Wiring', 'contact@nanomaterials.com'),
    ('OptiGlass', 'Optical Components', 'support@optiglass.com'),
    ('ThermalX', 'Thermal Paste', 'contact@thermalx.com'),
    ('FlexCircuits', 'Flexible Circuits', 'info@flexcircuits.com')
]

supply_chain_data = [
    (1, 1, '2023-12-20', 10000),
    (2, 2, '2023-12-25', 15000),
    (3, 3, '2023-12-30', 12000),
    (4, 4, '2024-01-05', 8000),
    (5, 5, '2024-01-10', 5000),
    (6, 6, '2024-01-15', 2000)
]

departments_data = [
    ('R&D', 'Santa Clara'),
    ('Sales', 'New York'),
    ('Operations', 'Austin'),
    ('Marketing', 'Los Angeles'),
    ('HR', 'Seattle')
]

employees_data = [
    ('Alice', 'Johnson', 1, '2020-06-15', 120000),
    ('Bob', 'Smith', 2, '2021-03-10', 85000),
    ('Charlie', 'Brown', 3, '2019-11-20', 95000)
]

projects_data = [
    ('AI Chip Development', 1, '2022-01-01', '2024-01-01', 5000000),
    ('Global Sales Expansion', 2, '2023-01-01', '2025-01-01', 2000000),
    ('Supply Chain Optimization', 3, '2021-01-01', '2023-12-31', 1000000)
]

In [5]:
# 3. Insert data into tables
cursor.executemany('INSERT INTO products (product_name, category, release_date, price) VALUES (?, ?, ?, ?)', products_data)
cursor.executemany('INSERT INTO customers (customer_name, industry, contact_email, contact_phone) VALUES (?, ?, ?, ?)', customers_data)
cursor.executemany('INSERT INTO sales (product_id, customer_id, sale_date, region, quantity_sold, revenue) VALUES (?, ?, ?, ?, ?, ?)', sales_data)
cursor.executemany('INSERT INTO suppliers (supplier_name, material_supplied, contact_email) VALUES (?, ?, ?)', suppliers_data)
cursor.executemany('INSERT INTO supply_chain (supplier_id, product_id, supply_date, quantity_supplied) VALUES (?, ?, ?, ?)', supply_chain_data)
cursor.executemany('INSERT INTO departments (department_name, location) VALUES (?, ?)', departments_data)
cursor.executemany('INSERT INTO employees (first_name, last_name, department_id, hire_date, salary) VALUES (?, ?, ?, ?, ?)', employees_data)
cursor.executemany('INSERT INTO projects (project_name, department_id, start_date, end_date, budget) VALUES (?, ?, ?, ?, ?)', projects_data)

In [6]:
# Commit changes and close the connection
conn.commit()
conn.close()

print("Database populated successfully!")

Database populated successfully!


In [7]:
import os
print(os.getcwd())

C:\Users\imaja\AppData\Local\Programs\Microsoft VS Code
